# Perform forward kinematics with a trajectory

In this notebook, we are going to generate a short video for a random robot movement you can watch in your browser.

First of all, we need some modules. You can create some simple modules built from geometric primitives yourself - there is a distinct tutorial to that purpose. If you don't want to, we can use the set of simple modules provided with the timor source code:

In [1]:
from pathlib import Path

# provide your own filepaths if you already did the module generation tutorial:
your_modules_json = Path('')
your_assets_directory = Path('')

if your_modules_json == Path(''):
    from timor.utilities.file_locations import get_module_db_files
    modules_file = get_module_db_files('geometric_primitive_modules')

2024-10-30 13:09:02,355 Timor INFO Loading custom configurations from /home/calix/.config/timor.config


In [2]:
import numpy as np
import timor
from timor.Module import *
from timor.utilities.visualization import animation, MeshcatVisualizerWithAnimation

#create the DB object
db = ModulesDB.from_json_file(modules_file)

#print out available modules in DB
print(db.by_name)
print(db.by_id)
# print(db.all_joints)
# print(db.all_connectors)


{'I shaped link 0.08-0.08-45': <timor.Module.AtomicModule object at 0x78c1e4b190f0>, 'I shaped link 0.08-0.08-15': <timor.Module.AtomicModule object at 0x78c1e4b19f90>, 'L shaped link 0.1-0.08-45': <timor.Module.AtomicModule object at 0x78c1e4b2c370>, 'I shaped link 0.08-0.08-30': <timor.Module.AtomicModule object at 0x78c1e4b19540>, 'L shaped link 0.1-0.08-30': <timor.Module.AtomicModule object at 0x78c1e4b1bb20>, 'Revolute Joint': <timor.Module.AtomicModule object at 0x78c1e4b18a60>, 'Base': <timor.Module.AtomicModule object at 0x78c1e4b1aa40>, 'Prismatic Joint': <timor.Module.AtomicModule object at 0x78c1e4b182e0>, 'L shaped link 0.1-0.08-15': <timor.Module.AtomicModule object at 0x78c1e4b1b310>, 'Demo EEF': <timor.Module.AtomicModule object at 0x78c1e4b18fa0>}
{'i_45': <timor.Module.AtomicModule object at 0x78c1e4b190f0>, 'i_15': <timor.Module.AtomicModule object at 0x78c1e4b19f90>, 'l_45': <timor.Module.AtomicModule object at 0x78c1e4b2c370>, 'i_30': <timor.Module.AtomicModule obj

In [3]:
#contains IDs we care about. The order is arbitrary, but if we want to make a robot out of this (i.e. kinematic tree) we should put in order.
modules = ('base', 'J2', 'i_45', 'J2', 'J2', 'eef')
A = ModuleAssembly.from_serial_modules(db, modules)
robot = A.to_pin_robot() #convert to pinocchio robot

viz = robot.visualize()
viz.viewer.jupyter_cell(height=400) # generate a jupyter cell to visualize the robot

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [4]:
#arbitrary new joint state
q2 = np.array([np.pi, -np.pi/2, np.pi/2])


"""Forwards Kinematics (FK) and Forwards Dynamics (FD) methods belong to PinRobot object"""
  # FK https://timor-python.readthedocs.io/en/latest/autoapi/timor/Robot/index.html#timor.Robot.PinRobot.fk
  # FD https://timor-python.readthedocs.io/en/latest/autoapi/timor/Robot/index.html#timor.Robot.PinRobot.fd

# FK calculates the relative rigid body transform from the reference config at a given joint angle position. 
# You can tell it to update the collision hitbox and visual mesh if wanted. I don't think this robot can collide with itself.
print(f"Robot end effector coordinate transform at joint state {q2}", robot.fk(configuration = q2, collision = True, visual = True)) #default kind is tcp - tool center point
print(f"Robot joint coordinate transfom at joint state {q2}", robot.fk(configuration = q2, kind = 'joints', collision = True, visual = True))
print(f"Robot full frames coordinate transfom at joint state {q2}", robot.fk(configuration = q2, kind = 'full', collision = True, visual = True))


#visual just updates the state of the visual geometries, we have to actually visualize to see it
vis = robot.visualize()

#Current robot velocities
print(f"Current robot joint positions (configuration): {robot.configuration}")
print(f"Current robot joint velocities: {robot.velocities}")

# FD calculates joint accelerations (ddq) given joint config (q) and joint velocity (dq) 
print(robot.fd(tau = [1, 0, 0], motor_inertia = False, friction = True))
  

Robot end effector coordinate transform at joint state [ 3.14159265 -1.57079633  1.57079633] [[-0.    1.   -0.    0.5 ]
 [-0.   -0.   -1.   -0.19]
 [-1.   -0.    0.    0.8 ]
 [ 0.    0.    0.    1.  ]]
Robot joint coordinate transfom at joint state [ 3.14159265 -1.57079633  1.57079633] ([[ 1.    0.    0.    0.2 ]
 [-0.    1.    0.   -0.  ]
 [ 0.   -0.    1.    0.05]
 [ 0.    0.    0.    1.  ]], [[-0.   0.   1.   0.2]
 [-1.  -0.  -0.   0. ]
 [ 0.  -1.   0.   0.8]
 [ 0.   0.   0.   1. ]], [[-0.   1.  -0.   0.5]
 [-0.  -0.  -1.   0. ]
 [-1.  -0.   0.   0.8]
 [ 0.   0.   0.   1. ]])
Robot full frames coordinate transfom at joint state [ 3.14159265 -1.57079633  1.57079633] ([[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]], [[ 1.  0.  0.  0.]
 [ 0. -1. -0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  1.]], [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]], [[-0.   -0.    1.    0.05]
 [ 0.   -1.    0.    0.  ]
 [ 1.    0.    0.    0.05]
 [ 0.    0.    0.    1.  ]], [[-0.    0

In [5]:
def specific_pose_valid(robot, theta: List[float], task = None) -> bool:
	#perform FK on the theta list
	robot.fk(theta, visual = True, collision = True)
	
	self_collisions = robot.has_self_collision()
	collisions = False if task is None else robot.has_collisions(task, safety_margin=0) #may need to alter safety margin

	return not (collisions or self_collisions)

In [ ]:
"""Example of manually defined trajectory with Forwards Kinematics (pt 1)"""

modules = ('base', 'i_30', 'J2', 'J2', 'J2', 'i_30', 'eef')
B = ModuleAssembly.from_serial_modules(db, modules)
long_robot = B.to_pin_robot() #convert to pinocchio robot

q_0 = [np.pi, np.pi, np.pi] #start in this config with no collisions

long_robot.fk(q_0, collision = True, visual = True)

#the collisions() method exists, but requires the definition of a task. we look at collision pairs as a quick shortcut.
print("Self collision in this state: ", long_robot.has_self_collision())
long_robot.visualize_self_collisions() #nothing is highlighted b/c no self collisions

print("Specific pose valid: ", specific_pose_valid(long_robot, q_0))

q_1 = [0,0,0]
long_robot.fk(q_1, collision = True, visual = True)

#the collisions() method exists, but requires the definition of a task. we look at collision pairs as a quick shortcut.
print("Self collision in this state: ", long_robot.has_self_collision())
long_robot.visualize_self_collisions() #nothing is highlighted b/c no self collisions

print("Specific pose valid: ", specific_pose_valid(long_robot, q_1))

/home/calix/miniforge3/envs/timor_test_env/lib/python3.10/site-packages/timor/Robot.py:820: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.linalg.det(J @ J.T))


nan
Self collision in this state:  False
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7006/static/
Specific pose valid:  True
Self collision in this state:  True
collision pair detected: 1 , 11 - collision: Yes
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/
Specific pose valid:  False


In [42]:
def find_trajectory(robot, base_config, target_config, task = None, max_rrt_iters: int = 10000, rrt_step_size: float = 0.1, target_distance_thresh: float = 0.2):
	'''
		Checks if a target pose is reachable from a base pose through any valid trajectory. 
		Returns a valid trajectory if found. Does not guarantee anything close to optimality.
		
		This implementation uses Rapidly-exploring Random Trees (RRT).

		IMPORTANT - this function works entirely within the joint space. no calculation of actual end effector position is done. this matters for the target distance threshold!

		Parameters:
			- robot
			- target_config
			- base_config
			- task
			- max_rrt_iters
			- rrt_step_size
			- target_distance_thresh

		Returns:
			- the whole tree (for now) if a trajectory exists. None otherwise

		TODO:
			- make the tree an actual tree - this impl is a little scuffed
	'''

	#if the base or target poses are invalid, no trajectory is possible.	
	if not specific_pose_valid(robot, base_config, task) or not specific_pose_valid(robot, target_config, task):
		return None

	#each node in the tree contains a tuple of (joint_angles: List[float], parent: Integer) - parent is the parent index 
	tree = [base_config]
	idx_to_parent = dict()
	idx_to_parent[0] = -1

	#nx2 arr - of lower and upper angle limits for each joint
	joint_space_bounds = robot.joint_limits
	low_bounds, high_bounds = joint_space_bounds[0], joint_space_bounds[1]

	#RRT Algo
	for _ in range(max_rrt_iters):
		# Sample random configuration
		random_config = np.random.uniform(low_bounds, high_bounds)
		
		# Find nearest node in the tree
		nearest_config_idx, nearest_config = min(enumerate(tree), key=lambda x: np.linalg.norm(x[1] - random_config))
		
		# Extend towards random config
		new_config = nearest_config + rrt_step_size * (random_config - nearest_config) / np.linalg.norm(random_config - nearest_config)
		#if the new config that we step into is valid, add it to the tree. we can 
		if specific_pose_valid(robot, new_config, task):
			# print(new_config, len(tree))
			#append a new config and its parent's idx in the tree
			idx_to_parent[len(tree)] = nearest_config_idx
			tree.append(new_config)
			
			# Check if we've reached the target. if so, return
			if np.linalg.norm(new_config - target_config) < target_distance_thresh:
				full_path = [target_config]

				next_node, parent_idx = new_config, nearest_config_idx

				while parent_idx != -1:
					full_path.insert(0, next_node)
					next_node, parent_idx = tree[parent_idx], idx_to_parent[parent_idx]

				return full_path
	
	#no trajectories are valid. return None
	return None

In [ ]:
viz = robot.visualize()
viz.viewer.jupyter_cell(height=400) # generate a jupyter cell to visualize the robot

traj = find_trajectory(robot, [0, 0, 0], [np.pi/2, np.pi, np.pi/2], target_distance_thresh = 1)

if traj:
    traj = np.array(traj)
    print(f"Found trajectory of length {traj.shape[0]}: \n{traj}")
    animation(robot, traj, .1)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7031/static/


AttributeError: 'list' object has no attribute 'shape'